# dogoGPT Training


## GPU Check


In [ ]:
!nvidia-smi

## Setup


In [ ]:
!git clone https://github.com/Kinyugo/dogoGPT.git 

In [ ]:
%cd dogoGPT
%pip install -q -r notebook_requirements.txt

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt # shakespeare
!wget https://raw.githubusercontent.com/Kinyugo/Walks_Into_A_Bar_Language_Model/master/data/walks_into_a_bar.txt -O bar_jokes.txt # bar jokes

## Imports


In [ ]:
import torch
from omegaconf import OmegaConf

from dogogpt.configs import TrainingConfig
from dogogpt.language_model import LanguageModel
from dogogpt.training import run_training


## Training


In [ ]:
dict_config = {
    "datamodule": {
        "src_path": "shakespeare.txt",
        "num_chars": 256,
        "batch_size": 32,
        "num_workers": 2,
        "pin_memory": True,
    },
    "lit_language_model": {"sample_every_n_steps": 1000},
    "trainer": {"max_steps": 10001},
}


In [ ]:
default_config = OmegaConf.structured(TrainingConfig)
custom_config = OmegaConf.create(dict_config)
config = OmegaConf.merge(default_config, custom_config)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

In [ ]:
run_training(config)


## Sampling


In [ ]:
ckpt_path = ""
device = "cuda" if torch.cuda.is_available() else "cpu"
lm = LanguageModel.from_pretrained(ckpt_path, device)


In [ ]:
prompts = ["\n"]
samples = lm(
    prompts,
    num_tokens=512,
    context_size=256,
    temperature=1.0,
    top_k=None,
    num_parallel_tokens=1,
    verbose=True,
)


In [ ]:
separator = "-" * 20
print(f"\n{separator}\n".join(samples))
